In [ ]:
import fiftyone as fo
import fiftyone.utils.coco as fouc
from fiftyone import ViewField as F

In [ ]:
# name = "dental_rtg_merged_full"
name = 'dental_rtg_merged3'
data_path = "/home/kuntik/dev/datasets/dental_merged/images"
# labels_path = "/home/kuntik/dev/datasets/dental_merged/annotations.json"
labels_path = "/home/kuntik/dev/datasets/dental_rtg_merged2/annotations.json"
dataset = fo.Dataset.from_dir(
    dataset_type = fo.types.COCODetectionDataset,
    data_path = data_path,
    labels_path = labels_path,
    name=name
)
# dataset = fo.load_dataset('dental_rtg_merged_f')
# dataset = fo.load_dataset('dental_caries_dataset_merged2')


In [ ]:
dataset = fo.load_dataset('foo3')

In [ ]:
print(dataset)
session = fo.launch_app(dataset,desktop=True)

Exception in thread Thread-19:
Traceback (most recent call last):
  File "/home/kuntik/miniconda3/envs/fiftyone/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/kuntik/miniconda3/envs/fiftyone/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kuntik/miniconda3/envs/fiftyone/lib/python3.8/site-packages/fiftyone/core/client.py", line 125, in run_client
    io_loop.run_sync(connect)
  File "/home/kuntik/miniconda3/envs/fiftyone/lib/python3.8/site-packages/tornado/ioloop.py", line 530, in run_sync
    return future_cell[0].result()
  File "/home/kuntik/miniconda3/envs/fiftyone/lib/python3.8/site-packages/fiftyone/core/client.py", line 92, in connect
    message = json_util.loads(message)
  File "/home/kuntik/miniconda3/envs/fiftyone/lib/python3.8/site-packages/bson/json_util.py", line 439, in loads
    return json.loads(s, *args, **kwargs)
  File "/home/kuntik/miniconda3/envs/fiftyone/lib/python3


Session disconnected, trying to reconnect


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds



In [ ]:
session.view=dataset.view()

In [ ]:
# dataset.take()
# dataset.filter_labels('predictions', F("confidence") > 0.15).evaluate_detections(
results = dataset.evaluate_detections(
    "new_predictions",
    gt_field="ground_truth",
    iou=0.5,
    compute_mAP = True,
    # eval_key="eval"
)

In [ ]:
pred_name = 'new_predictions'
val_v = dataset.match_tags('val')
val_v15 = val_v.filter_labels(pred_name, F('confidence')>0.30)
results = val_v15.evaluate_detections(
    pred_name,
    gt_field="ground_truth",
    iou=0.5,
    compute_mAP = True
)
results.print_report()

results.print_report()
results.mAP()

session.view = dataset.sort_by('eval_fn', reverse=True)

In [ ]:
dataset_15 = dataset.filter_labels('new_predictions', F("confidence") > 0.15)
dataset_15 = dataset.evaluate_detections(
    "new_predictions",
    gt_field = "ground_truth",
    iou=0.5,
    compute_mAP = True,
    eval_key='eval_new'
)

In [ ]:
import fiftyone.brain as fob
fob.compute_mistakenness(
    supressed_view15,
    'predictions',
    label_field='ground_truth',
    use_logits=False
)

In [ ]:
anno_key = "p3"

label_schema = {
    "ground_truth" : {
        "type" : "detections",
        "classes" : ["decay"]
    },
    "predictions" : {
        "type" : "detections",
        "classes" : ["decay"]
    }
}

# Upload the samples and launch CVAT
annot_res = dataset[1800:].annotate(
    anno_key=anno_key,
    label_schema=label_schema,
    attributes=False,
    image_quality=100,
    segment_size=100
)

In [ ]:
import json
from pathlib import Path
with open('rtg2_preds_effdet.json', 'r') as json_file:
    preds = json.load(json_file)
# dataset['/home/kuntik/carries_dataset/images/1.png']
data_path = '/home/kuntik/dev/datasets/dental_merged/images'
add_prediction(preds, dataset, Path(data_path))

In [ ]:
from PIL import Image
def add_prediction(pred_dict, dataset, img_root):
    # for sample in dataset:
    det_id = 0
    for pred_key, pred_value in pred_dict.items():
        dets = []
        # id = sample.id
        path = img_root / pred_key
        sample = dataset[str(path)]
        s_id = sample.id
        scores = pred_value['scores']
        labels = pred_value['labels']
        bboxes = pred_value['bboxes']
        img = Image.open(path)
        for box, score, label in zip(bboxes, scores, labels):
            #TODO to limit confidence
            # if score < 0.22:
            #     continue
            coco_obj = fouc.COCOObject(det_id, s_id, int(label), [box[0],box[1], box[2]-box[0], box[3]-box[1]], score=score)
            det = coco_obj.to_detection(img.size, ['decay'])
            dets.append(det)
            det_id += 1
        detection = fo.Detections(detections=dets)
        sample['predictions'] = detection
        # sample.tags.append(pred_value['stage'])
        sample.tags = [pred_value['stage']]
        sample.save()
        

